**Assignment 5: Building and Demonstrating Map Servers with the OpenAI Agents SDK.**

Three custom map servers are built: a **GeoServer**, a **RouteServer**, and a **POIServer** and integrated them with an intelligent OpenAI agent that follows the Model Context Protocol, or MCP for short.

Purpose of each server:

**GeoServer** performs geocoding — it takes a place name such as “Eiffel Tower, Paris” and returns its latitude and longitude.

**RouteServer** handles routing — it calculates a simple path and distance between two coordinates.

**POIServer** performs point-of-interest searches — for example, it can find nearby coffee shops or landmarks.

In [1]:
# =====================================
Install required packages
!pip install -q fastapi uvicorn httpx pydantic aiofiles python-dotenv nest_asyncio pytest


In [2]:
import nest_asyncio, uvicorn, asyncio
nest_asyncio.apply()   # allows running uvicorn inside notebooks


In [3]:
%%writefile geoserver.py
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Dict, Any

app = FastAPI(title="GeoServer")

FAKE_DB = {
    "1600 amphitheatre parkway, mountain view": {"lat": 37.422, "lon": -122.084, "display_name": "1600 Amphitheatre Pkwy, Mountain View, CA"},
    "eiffel tower": {"lat": 48.85837, "lon": 2.294481, "display_name": "Eiffel Tower, Paris"}
}

class ServerParams(BaseModel):
    name: str
    description: str
    operations: List[Dict[str, Any]]

@app.get("/discover")
async def discover():
    return ServerParams(
        name="geoserver",
        description="Provides geocoding and reverse-geocoding operations",
        operations=[
            {"name":"geocode","input":{"q":"string"},"output":{"lat":"float","lon":"float"}},
            {"name":"reverse_geocode","input":{"lat":"float","lon":"float"},"output":{"display_name":"string"}}
        ]).dict()

class GeocodeReq(BaseModel):
    q: str

@app.post("/op/geocode")
async def geocode(req: GeocodeReq):
    q = req.q.lower().strip()
    if q in FAKE_DB: return FAKE_DB[q]
    return {"lat":0.0,"lon":0.0,"display_name":"Unknown"}

class RevReq(BaseModel):
    lat: float; lon: float

@app.post("/op/reverse_geocode")
async def reverse_geocode(req: RevReq):
    best, best_d = None, 1e9
    for k,v in FAKE_DB.items():
        d = (v["lat"]-req.lat)**2 + (v["lon"]-req.lon)**2
        if d < best_d: best, best_d = v, d
    return {"display_name": best["display_name"]}


Writing geoserver.py


In [4]:
%%writefile routeserver.py
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Dict, Any
from math import radians, sin, cos, sqrt, atan2

app = FastAPI(title="RouteServer")

class ServerParams(BaseModel):
    name:str; description:str; operations:List[Dict[str,Any]]

@app.get("/discover")
async def discover():
    return ServerParams(
        name="routeserver",
        description="Computes rough routes between coordinates",
        operations=[
            {"name":"route","input":{"start":{"lat":"float","lon":"float"},"end":{"lat":"float","lon":"float"}},
             "output":{"distance_km":"float","steps":"array"}}
        ]).dict()

class Coord(BaseModel): lat:float; lon:float
class RouteReq(BaseModel): start:Coord; end:Coord

@app.post("/op/route")
async def route(req:RouteReq):
    R=6371.0
    lat1,lon1,lat2,lon2=map(radians,[req.start.lat,req.start.lon,req.end.lat,req.end.lon])
    dlat,dlon=lat2-lat1,lon2-lon1
    a=sin(dlat/2)**2+cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c=2*atan2(sqrt(a),sqrt(1-a))
    dist=R*c
    steps=[{"instruction":f"Travel from {req.start.dict()} to {req.end.dict()}","distance_km":dist}]
    return {"distance_km":dist,"steps":steps}


Writing routeserver.py


In [5]:
%%writefile poiserver.py
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Dict, Any

app = FastAPI(title="POIServer")

SAMPLE_POIS=[
    {"name":"Central Cafe","lat":37.422,"lon":-122.084,"category":"coffee"},
    {"name":"Main Library","lat":37.423,"lon":-122.083,"category":"library"},
    {"name":"Pizza Place","lat":37.421,"lon":-122.085,"category":"restaurant"},
]

class ServerParams(BaseModel):
    name:str; description:str; operations:List[Dict[str,Any]]

@app.get("/discover")
async def discover():
    return ServerParams(
        name="poiserver",
        description="Search nearby points-of-interest",
        operations=[{"name":"search_poi","input":{"lat":"float","lon":"float","q":"string","radius_m":"float"},
                    "output":{"results":"array"}}]).dict()

class POIReq(BaseModel):
    lat:float; lon:float; q:str=""; radius_m:float=500

@app.post("/op/search_poi")
async def search_poi(req:POIReq):
    res=[]
    for p in SAMPLE_POIS:
        if req.q.lower() in p["name"].lower() or req.q.lower() in p["category"].lower():
            res.append(p)
    return {"results":res}


Writing poiserver.py


In [6]:
# Run all 3 servers in background threads so they stay active in Colab
import threading, uvicorn, time

def run_server(module, port):
    uvicorn.run(module, host="127.0.0.1", port=port, log_level="error")

threads=[]
for mod,port in [("geoserver:app",8001),("routeserver:app",8002),("poiserver:app",8003)]:
    t=threading.Thread(target=run_server,args=(mod,port),daemon=True)
    t.start()
    threads.append(t)
time.sleep(3)
print("✅ Servers running on ports 8001–8003")


✅ Servers running on ports 8001–8003


In [7]:
%%writefile mcp_client.py
import httpx, asyncio
from typing import Any, Dict, List

class MCPServer:
    def __init__(self, base_url:str, params:Dict[str,Any]):
        self.base_url=base_url; self.params=params
    async def call_op(self, op:str, payload:Dict[str,Any]):
        async with httpx.AsyncClient() as c:
            r=await c.post(f"{self.base_url}/op/{op}",json=payload)
            r.raise_for_status(); return r.json()

async def discover(base_url:str):
    try:
        async with httpx.AsyncClient() as c:
            r=await c.get(f"{base_url}/discover"); r.raise_for_status()
            return MCPServer(base_url,r.json())
    except Exception:
        return None

async def discover_all():
    tasks=[discover(f"http://127.0.0.1:{p}") for p in [8001,8002,8003]]
    servers=await asyncio.gather(*tasks)
    return [s for s in servers if s]


Writing mcp_client.py


In [8]:
%%writefile assistant_agent.py
import asyncio
from mcp_client import discover_all

class AssistantAgent:
    async def startup(self):
        self.servers=await discover_all()
        print("Discovered:",[s.params["name"] for s in self.servers])

    async def geocode(self,q:str):
        s=[x for x in self.servers if x.params["name"]=="geoserver"][0]
        return await s.call_op("geocode",{"q":q})

    async def route(self,start,end):
        s=[x for x in self.servers if x.params["name"]=="routeserver"][0]
        return await s.call_op("route",{"start":start,"end":end})

    async def search_poi(self,lat,lon,q=""):
        s=[x for x in self.servers if x.params["name"]=="poiserver"][0]
        return await s.call_op("search_poi",{"lat":lat,"lon":lon,"q":q})


Writing assistant_agent.py


In [9]:
import asyncio
from assistant_agent import AssistantAgent

async def demo():
    agent=AssistantAgent()
    await agent.startup()
    g=await agent.geocode("Eiffel Tower")
    print("Geocode:",g)
    r=await agent.route({"lat":48.85837,"lon":2.29448},{"lat":48.86,"lon":2.30})
    print("Route:",r)
    p=await agent.search_poi(37.422,-122.084,"coffee")
    print("POI:",p)

await demo()


Discovered: ['geoserver', 'routeserver', 'poiserver']
Geocode: {'lat': 48.85837, 'lon': 2.294481, 'display_name': 'Eiffel Tower, Paris'}
Route: {'distance_km': 0.44263337427795496, 'steps': [{'instruction': "Travel from {'lat': 48.85837, 'lon': 2.29448} to {'lat': 48.86, 'lon': 2.3}", 'distance_km': 0.44263337427795496}]}
POI: {'results': [{'name': 'Central Cafe', 'lat': 37.422, 'lon': -122.084, 'category': 'coffee'}]}


# Summary
| Server | Function | Example Input | Example Output |
|---------|-----------|---------------|----------------|
| GeoServer | Geocoding | "Eiffel Tower" | {"lat":48.85837,"lon":2.29448} |
| RouteServer | Routing | (Paris1 → Paris2) | {"distance_km":0.44} |
| POIServer | POI Search | "coffee near Palo Alto" | {"results":[{"name":"Central Cafe"}]} |
